## 6.1 멀티 프로세스와 멀티 스레드

- 운영 체제에서 실행되는 각 프로그램 = 각각이 별도의 `프로세스`.
- 각 프로세스에는 하나 이상의 `스레드`가 있다.

`멀티 프로세스`: 별도의 메모리 영역을 가지며, 특별한 메커지늠으로만 통신할 수 있다. `프로세서`는 각 스레드에 대해 별도의 레지스터 집합을 불러오거나 저장, 프로세스간 데이터 공유와 통신용으로는 비효율적. 파이썬에서는 `subprocess` 사용

`멀티 스레드`: 단일 프로세스 내의 멀티 스레드는 동일한 메모리에 접근. 스레드는 데이터 공유를 통해 간단하게 통신한다. `threading` 모듈의 처리를 통해 한번에 한 스레드만 메모리 영역에 접근할 수 있다. 각 프로세스가 독립적인 `stack`, `heap`, `code`, `data` 를 가지는 반면, 한 프로세스에 속한 스레드는 `스택` 영역을 제외한 메모리 영역 공유.

- 멀티 스레드에서 한 스레드는 `인터프리터 락`을 획득하여야 실행될 수 있다.  인터프리터 락은 하나만 존재, 이것을 `GIL`이라고 함. GIL을 획득한 스레드가 어떤 자원을 사용시, 다른 스레드들은 이 자원에 접근할 수 없다.

`동시성`: 논리적으로 여러 작업이 동시에 실행되는 것처럼 보이는 것. 예를들어 I/O(파일 및 네트워크 소켓 입력 및 출력, 데이터를 주고받는 연산) 연산 등은 프로그램의 흐름에 큰 짐이 될 수 있다. 한 작업의 I/O 연산이 완료되기를 기다리는 동안 다른 작업을 수행하여 `유휴시간을 잘 활용`하는것이 `동시성`

`병렬성`: 물리적으로 여러 작업이 동시에 처리되는 것. 데이터 병렬성과 작업 병렬성으로 나뉨. 
`데이터 병렬성`: 같은 작업을 병렬처리 하는것. 
`작업 병렬성`: 서로 다른 작업을 병렬처리하는 것.
    - 웹 서버에서는 다수의 독립적인 요청을 병렬로 개별적으로 처리가능.

### 6.1.1 subprocess 모듈
- `subprocess` 모듈은 부모-자식 프로세스 쌍을 생성하는 데 사용됨. 
- 부모 프로세스는 사용자에 의해 실행
- 부모 프로세스는 차례로 다른 일을 처리하는 자식 프로세스의 인스턴스를 실행. 
- 자식 프로세스를 사용함으로써, 멀티 코어의 이점을 최대한 취하고, 동시성 문제를 운영 체제가 알아서 처리하도록 함

In [9]:
import subprocess
subprocess.run(["echo", "이것은 subprocess 입니다."])

CompletedProcess(args=['echo', '이것은 subprocess 입니다.'], returncode=0)

In [6]:
subprocess.run(["sleep", "10"]) #10초 동안 sleep

CompletedProcess(args=['sleep', '10'], returncode=0)

In [ ]:
#### 6.1.2 threading 모듈
- 스레드가 여러개로 분리되면, 스레드 간 데이터 공유의 복잡성이 증가.
- lock, deadklock을 회피하는데 주의를 기울여야 함.
- 파이썬에서는 단 하나의 메인 스레드만 존재
- 내부적으로 락을 관리하려면 `queue` 모듈 사용. 
- `스레드 데몬` 으로 변환하면, 워커 스레드가 작업완료시 종료되지 않고 계속 실행되는 문제 해결.
- queue.join()메서드는 큐가 빌때까지(큐의 모든 항목이 처리될 때까지_ 기다린다. 

In [27]:
import queue
import threading 

q = queue.Queue()

def worker(num):
    while True:
        item = q.get()
        if item is None:
            break
        # 작업을 처리한다.
        print("스레드 {0}: 처리 완료 {1}".format(num+1, item))
        q.task_done()
        
if __name__ == "__main__":
    num_worker_threads = 5
    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker, args=(i,))
        t.start()
        threads.append(t)
    
    for item in range(20):
        q.put(item)
        
    #모든 작업이 끝날 때까지 대기한다(block).
    q.join()
    
    # 워커 스레드를 종료한다(stop).
    for i in range(num_worker_threads):
        q.put(None)
    
    for t in threads:
        t.join()

스레드 1: 처리 완료 0스레드 4: 처리 완료 1스레드 5: 처리 완료 2스레드 2: 처리 완료 3스레드 3: 처리 완료 4


스레드 3: 처리 완료 5스레드 4: 처리 완료 6
스레드 5: 처리 완료 7

스레드 4: 처리 완료 8
스레드 4: 처리 완료 9
스레드 4: 처리 완료 10
스레드 4: 처리 완료 11
스레드 4: 처리 완료 12
스레드 4: 처리 완료 13
스레드 4: 처리 완료 14
스레드 4: 처리 완료 15
스레드 4: 처리 완료 16

스레드 4: 처리 완료 17
스레드 4: 처리 완료 18
스레드 4: 처리 완료 19



#### 6.1.3 뮤텍스와 세마포어

- `뮤텍스`는 락과 같다. 
- 공유 리소스에 한번에 하나의 스레드만 접근할 수 있도록 하는 상호배제 동시성 제어 정책을 강제

In [22]:
from threading import Thread, Lock
import threading

def worker(mutex, data, thread_safe):
    if thread_safe:
        mutex.acquire()
    try:
        print("스레드 {0}: {1}\n".format(threading.get_ident(), data))
        
    finally:
        if thread_safe:
            mutex.release()

if __name__ == "__main__":
    threads = []
    thread_safe = True
    mutex = Lock()
    
    for i in range(20):
        t = Thread(target=worker, args=(mutex, i, thread_safe))
        # target = 스레드에서 실행할 함수
        # args =  target에 넘겨질 인자.
        # daemon = 데몬 실행여부, 데몬으로 실행되는 스레드는 프로세스가 종료될 때 즉각 중단된다.
        t.start()
        threads.append(t)
        
    for t in threads:
        t.join()
        
    # start() = 스레드를 시작한다
    # join() = 해당 스레드에서 실행되는 함수가 종료될때까지 기다린다. (워커 스레드들이 동작하고 있는 중일 때 /
    # 메인스레드가 적절히 기다려주어야함)
    # ident = 스레드 식별자. 정수값

# 참고: https://soooprmx.com/archives/8834

스레드 123145347305472: 0

스레드 123145347305472: 1

스레드 123145352560640: 2

스레드 123145347305472: 3

스레드 123145352560640: 4

스레드 123145347305472: 5

스레드 123145352560640: 6

스레드 123145347305472: 7

스레드 123145352560640: 8

스레드 123145347305472: 9

스레드 123145347305472: 10

스레드 123145352560640: 11

스레드 123145357815808: 12

스레드 123145347305472: 13

스레드 123145352560640: 14

스레드 123145357815808: 15

스레드 123145347305472: 16

스레드 123145352560640: 17

스레드 123145357815808: 18

스레드 123145347305472: 19



#### 세마포어
- 세마포어는 1도다 큰 수로 시작할 수 있다.
- 세마포어값은 곧 한 번에 자원에 접근할 수 있는 스레드의 수 이다.
- 뮤텍스의 락 및 언락 작업과 유사한 `대기(wait)` 및 `신호(signal)` 작업 지원

In [6]:
import threading
import time

class ThreadPool(object):
    def __init__(self):
        self.active = []
        self.lock = threading.Lock()
        
    def acquire(self, name):
        with self.lock:
            self.active.append(name)
            print("획득: {0} | 스레드 풀: {1}".format(name, self.active))
            
    def release(self, name):
        with self.lock:
            self.active.remove(name)
            print("반환: {0} | 스레드 풀: {1}".format(name, self.active))
            
def worker(semaphore, pool):
    with semaphore:
        name = threading.currentThread().getName()
        pool.acquire(name)
        time.sleep(1)
        pool.release(name)

if __name__ == "__main__":
    threads = []
    pool = ThreadPool()
    semaphore = threading.Semaphore(3)
    for i in range(10):
        t = threading.Thread(target=worker, name="스레드 " + str(i), args=(semaphore, pool))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

획득: 스레드 0 | 스레드 풀: ['스레드 0']
획득: 스레드 1 | 스레드 풀: ['스레드 0', '스레드 1']
획득: 스레드 2 | 스레드 풀: ['스레드 0', '스레드 1', '스레드 2']
반환: 스레드 0 | 스레드 풀: ['스레드 1', '스레드 2']
획득: 스레드 3 | 스레드 풀: ['스레드 1', '스레드 2', '스레드 3']
반환: 스레드 1 | 스레드 풀: ['스레드 2', '스레드 3']
획득: 스레드 4 | 스레드 풀: ['스레드 2', '스레드 3', '스레드 4']
반환: 스레드 2 | 스레드 풀: ['스레드 3', '스레드 4']
획득: 스레드 5 | 스레드 풀: ['스레드 3', '스레드 4', '스레드 5']
반환: 스레드 3 | 스레드 풀: ['스레드 4', '스레드 5']
획득: 스레드 6 | 스레드 풀: ['스레드 4', '스레드 5', '스레드 6']
반환: 스레드 4 | 스레드 풀: ['스레드 5', '스레드 6']
획득: 스레드 7 | 스레드 풀: ['스레드 5', '스레드 6', '스레드 7']
반환: 스레드 5 | 스레드 풀: ['스레드 6', '스레드 7']
획득: 스레드 8 | 스레드 풀: ['스레드 6', '스레드 7', '스레드 8']
반환: 스레드 6 | 스레드 풀: ['스레드 7', '스레드 8']
획득: 스레드 9 | 스레드 풀: ['스레드 7', '스레드 8', '스레드 9']
반환: 스레드 7 | 스레드 풀: ['스레드 8', '스레드 9']
반환: 스레드 8 | 스레드 풀: ['스레드 9']
반환: 스레드 9 | 스레드 풀: []


#### 6.1.4 데드락과 스핀락
- 데드락은 두개 이상의 프로세스나 스레드가 서로 상대방의 작업이 끝나기만을 기다리고 있기 때문에 결과적으로 아무것도 완료되지 못하는 상태
- 프로그램에서 락을 할당하고, 락을 순서대로 획득시, 교착상태 막을 수 있다.

- 막을 수 있는 방법 4가지
    1. 상호배제(mutual exclusion): 자원은 한번에 한 프로세스만 사용할 수 있다.
    2. 점유와 대기(hold and wait): 한 프로세스가 자원을 가지고 있는 상태에서, 다른 포르세스가 쓰는 자원의 반납을 기다린다.
    3. 비선점(no preemption): 다른 프로세스가 이미 점유한 자원을 강제로 뺏어오지 못함
    4. 순환대기(circular wait): 프로세스 A,B,C가 있다고 가정할 때 A는 B가 점유한 자원을, B는 C가 점유한 자원을\
    , C는 A가 점유한 자원을 대기하는 상태

- `스핀락(spinlock)` : 고성능 컴퓨팅 상황에 유용한 바쁜대기(특정 공유 자원에 대해 두개 이상의 프로세스나 스레드가 그 이용 권한을 획득하기 위해 대기하는 현상)의 한 형태, 진입이 가능할 때까지 반복문을 돌면서 재시도 하는 방식

#### 6.1.5 스레딩에 대한 구글 파이썬 스타일 가이드
- 내장 타입의 원자성(atomicity)에 의존하지 않는다. 
- 딕셔너리 같은 파이썬 기본 타입은 원자적 연산을 수행하는 반면, 내장 타입이 원자적이지 않은 경우가 있어, 내장 타입의 원자성에 의존해선 안된다. 

In [31]:
import threading

def consumer(cond):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:
        print("{0} 대기".format(name))
        cond.wait()
        print("{0} 자원소비".format(name))
        
def producer(cond):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:
        print("{0} 자원 생산 후 모든 소비자에게 알림".format(name))
        cond.notifyAll()
        
if __name__ == "__main__":
    condition = threading.Condition()
    consumer1 = threading.Thread(
        name = "소비자1" ,target=consumer, args=(condition,))
    consumer2 = threading.Thread(
        name = "소비자2" ,target=consumer, args=(condition,))
    producer = threading.Thread(name="생산자", target=producer, args=(condition,))
    
    consumer1.start()
    consumer2.start()
    producer.start()

소비자1 시작
소비자1 대기
소비자2 시작
소비자2 대기
생산자 시작
생산자 자원 생산 후 모든 소비자에게 알림
소비자2 자원소비
소비자1 자원소비


#### 6.2.3 프로파일링
- 프로그램이 매우 느리게 실행되거나 예상보다 많은 메모리가 소비된다면, \
자료구조나 알고리즘을 잘못 선택했거나 비효율적으로 구현했기떄문

- 읽기전용 데이터는 리스트 대신 튜플 사용
- 반복문에서 항목이 많은 리스트나 튜플 대신, 제너레이터를 사용하여 순회

In [50]:
employee_list = [("Youn", "Kangyeol")]

In [51]:
# 좋은 예

items = ["<table>"]
for last_name, first_name in employee_list:
    items.append("<tr><td>%s, %s</td></tr>" %(last_name, first_name))
    items.append("</table>")
    print(items)
employee_table = "".join(items)

['<table>', '<tr><td>Youn, Kangyeol</td></tr>', '</table>']


In [52]:
employee_table

'<table><tr><td>Youn, Kangyeol</td></tr></table>'

In [2]:
import queue
import threading 
import time

q = queue.Queue()

def worker(num):
    while True:
        item = q.get()
        if item is None:
            print("스레드 {0}: None이 들어왔습니다. thread를 종료합니다".format(num+1))
            break
        # 작업을 처리한다.
        print("스레드 {0}: 시작".format(num+1, item))
        time.sleep(1)
        print("스레드 {0}: 처리 완료".format(num+1, item))
        q.task_done()
        
if __name__ == "__main__":
    num_worker_threads = 2
    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker, args=(i,))
        t.start()
        threads.append(t)
    
    for item in range(2):
        q.put(item)
        
    #모든 작업이 끝날 때까지 대기한다(block).
    q.join()
    print("Queue is Empty!")
    # 워커 스레드를 종료한다(stop).
    for i in range(num_worker_threads):
        q.put(None)
    
    for t in threads:
        t.join()
    print("Main Thread Done")

스레드 1: 시작스레드 2: 시작

스레드 1: 처리 완료스레드 2: 처리 완료

Queue is Empty!
스레드 1: None이 들어왔습니다. thread를 종료합니다스레드 2: None이 들어왔습니다. thread를 종료합니다

Main Thread Done
